In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
from utilities import clean_tweet
from tqdm.notebook import tqdm
tqdm.pandas()
import tweetnlp

def get_pred(model, tweet):
    ouput = model.predict(tweet)
    return ouput['label'] 


# Load models from tweetnlp
model_hate_speech = tweetnlp.load_model('hate')
model_offensive_language = tweetnlp.load_model('offensive')
model_sentiment = tweetnlp.load_model('sentiment')
model_topic = tweetnlp.load_model('topic_classification', multi_label=False) 

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# Read all parquet files from Tweets folder and concatenate them all into one dataframe
df_full = pd.concat([pd.read_parquet(f) for f in glob.glob('Tweets/*.parquet')], ignore_index = True)

In [6]:
# sample 1000 tweets
df = df_full.sample(1000)
df['Tweet_Clean'] = df['Tweet'].progress_apply(lambda x: clean_tweet(x))

# df['Tweet_Clean'] = df['Tweet'].progress_apply(lambda x: clean_tweet(x))
df['Sentiment'] = df['Tweet_Clean'].apply(lambda x: get_pred(model_sentiment, x))
df['Hate_Speech'] = df['Tweet_Clean'].progress_apply(lambda x: get_pred(model_hate_speech, x))
df['Offensive'] = df['Tweet_Clean'].apply(lambda x: get_pred(model_offensive_language, x))
df['Topic_Sport'] = df['Tweet_Clean'].apply(lambda x: get_pred(model_topic, x))


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [7]:
df.head()

,Datetime,Tweet_Id,Tweet,Username,Reply_Count,Retweet_Count,Like_Count,Bank,Tweet_Clean,Sentiment,Hate_Speech,Offensive,Topic_Sport
2550204,2014-03-28 07:06:27+00:00,449442375098957824,@CapitecBankSA I would pay d balance that I ow...,Lindimadiba,0,0,0,capitec,i would pay d balance that i owe to get my dip...,neutral,non-hate,non-offensive,business_&_entrepreneurs
889896,2018-10-02 08:17:43+00:00,1047037900620206081,Am I the only one struggling to log onto my @F...,Ms_Hloni,0,0,0,fnb,am i the only one struggling to log onto my app,negative,non-hate,non-offensive,science_&_technology
3204201,2020-09-21 12:01:05+00:00,1308013377223110657,"@busrep Definitely agree that the banks, and n...",ResDevFGroup,0,0,0,fnb,definitely agree that the banks and notably do...,negative,non-hate,non-offensive,business_&_entrepreneurs
3008857,2021-03-30 08:23:13+00:00,1376812242109394950,@Nedbank I managed to process a payment late a...,Asiphe50,1,0,0,nedbank,i managed to process a payment late afternoon ...,negative,non-hate,non-offensive,business_&_entrepreneurs
420307,2017-05-28 10:13:37+00:00,868772239562473472,@Orlando_Pirates You let us down this season b...,matshoba_nelson,0,0,1,nedbank,you let us down this season but we love you ju...,positive,non-hate,non-offensive,sports_&_gaming


In [8]:
df.to_csv("100_samples.csv", index=False)

In [ ]:
# df['Offensive'] = df['Tweet_Clean'].apply(lambda x: get_pred(model_offensive_language, x))

In [ ]:
# df['Topic_Sport'] = df['Tweet_Clean'].apply(lambda x: get_pred(model_topic, x))